In [ ]:
import numpy as np
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D

import os
print(os.listdir("../input"))

import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

In [ ]:
main_dir = "/kaggle/working/"
train_dir = "train"
path = os.path.join(main_dir,train_dir)

for p in os.listdir(path):
    category = p.split(".")[0]
    img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    new_img_array = cv2.resize(img_array, dsize=(80, 80))
    plt.imshow(new_img_array,cmap="gray")
    break

In [ ]:
X = []
y = []
convert = lambda category : int(category == 'dog')
def create_test_data(path):
    for p in os.listdir(path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)

In [ ]:
create_test_data(path)
X = np.array(X).reshape(-1, 80,80,1)
y = np.array(y)

In [ ]:
X = X/255.0

# Implementation with Keras

### **Types of layer in a CNN:**

1. Convolution
2. Pooling
3. Fully Connected

### Convolution Operation (Conv2D)

* The convulution operation is one of the fundemantal building a CNN.
* We have a input matrix(the input picture) and a filter(feature detector).
* Filter usally is a 3x3 matrix but it is not a rule.
* Filter detects horizantal or vertical lines and convex shape on the picture. For example in a person picture, we can find ears or noise etc.

## **Pooling Operation (MaxPooling2D)**

 We apply pooling to reduce the size of network and speed the computation. We can apply avarage pooling or max pooling. Let's suppose we have 4x4 input matrix, If we apply max pooling then the output will be 2x2 matrix. The way you do that is really simple. It has two hyperparameters, filter size(f) and stride(s).
 

![pooling](https://media.geeksforgeeks.org/wp-content/uploads/20190721025744/Screenshot-2019-07-21-at-2.57.13-AM.png)

### **Flattening (Flatten())**

Flattening is converting the output of convolutional layers into a 1 dimensional array for inputing  it to next layer. It is connected to fully connected layer.

###### ![flattening](https://miro.medium.com/max/1732/0*4afqrwiDydc8hw7g)

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Flatten())
model.add(Dense(64, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))


### *Optimizers*

Optimizers are used to change the attributes of a neural network such as weights and learning rate in order to reduce the losses. We will apply Adam optimizer.





In [ ]:
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]

)

In [ ]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
train_dir = "test1"
path = os.path.join(main_dir,train_dir)
#os.listdir(path)

X_test = []
id_line = []
def create_test1_data(path):
    for p in os.listdir(path):
        id_line.append(p.split(".")[0])
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_test.append(new_img_array)
create_test1_data(path)
X_test = np.array(X_test).reshape(-1,80,80,1)
X_test = X_test/255

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predicted_val = [int(round(p[0])) for p in predictions]

In [ ]:
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})

In [ ]:
submission_df.to_csv("submission.csv", index=False)